<a href="https://colab.research.google.com/github/Geeta80/DDOS_Detection/blob/master/multiclass_ovo_ovr_oner_cic_ddos2019_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
from google.colab import userdata
userdata.get('geetasingh19')

'7fe3ebc7b0534c06b65a6226b1701273'

In [4]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [5]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

geetasingh19_cicddos2019labelencoded_path = kagglehub.dataset_download('geetasingh19/cicddos2019labelencoded')
geetasingh19_cicddos2019only11catlabelbalanced_train_path = kagglehub.dataset_download('geetasingh19/cicddos2019only11catlabelbalanced-train')

print('Data source import complete.')


Data source import complete.


In [6]:
from fastcore.basics import *
from fastcore.parallel import *
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, roc_curve, precision_score, recall_score, f1_score, accuracy_score
from functools import partial
from os import cpu_count
import pandas as pd
import numpy as np

In [7]:
file_path = geetasingh19_cicddos2019only11catlabelbalanced_train_path + '/CICDDoS2019Only12CatLabelBalanced_train.csv'
df = pd.read_csv(file_path)

In [8]:
df.columns = df.columns.str.replace(' ', '')
df.head(10)

,Protocol,FlowDuration,TotalFwdPackets,TotalBackwardPackets,FwdPacketsLengthTotal,BwdPacketsLengthTotal,FwdPacketLengthMax,FwdPacketLengthMin,FwdPacketLengthMean,FwdPacketLengthStd,...,FwdSegSizeMin,ActiveMean,ActiveStd,ActiveMax,ActiveMin,IdleMean,IdleStd,IdleMax,IdleMin,Label
0,17,40694755,5,0,1500,0,300,300,300.00000,0.000000,...,-1062718975,0.000000,0.000000,0,0,10173689.0,3.593932e+06,14671724,6369775,Benign
1,0,113871580,52,0,0,0,0,0,0.00000,0.000000,...,0,6.083334,0.668558,7,5,9489292.0,2.133550e+05,9806952,9169328,Benign
2,0,112936309,39,0,0,0,0,0,0.00000,0.000000,...,0,4.916666,0.514929,6,4,9411354.0,2.927876e+05,9869393,9048974,Benign
3,6,24944828,8,6,37,0,31,0,4.62500,10.662852,...,20,20876.000000,313.955400,21098,20654,10009953.0,3.139554e+02,10010175,10009731,Benign
4,17,20779,2,2,86,366,43,43,43.00000,0.000000,...,0,0.000000,0.000000,0,0,0.0,0.000000e+00,0,0,Benign
5,6,118074966,36,48,2474,67854,517,0,68.72222,134.701700,...,20,88838.000000,15440.384000,99756,77920,58909704.0,2.302835e+04,58925988,58893420,Benign
6,17,20568,2,2,86,118,43,43,43.00000,0.000000,...,32,0.000000,0.000000,0,0,0.0,0.000000e+00,0,0,Benign
7,17,65675,2,2,86,118,43,43,43.00000,0.000000,...,32,0.000000,0.000000,0,0,0.0,0.000000e+00,0,0,Benign
8,17,20684,2,2,86,198,43,43,43.00000,0.000000,...,20,0.000000,0.000000,0,0,0.0,0.000000e+00,0,0,Benign
9,6,90699,4,2,62,70,31,0,15.50000,17.897858,...,20,0.000000,0.000000,0,0,0.0,0.000000e+00,0,0,Benign


In [9]:
df.FlowDuration.value_counts()

,count
FlowDuration,
1,55787
2,16863
48,1698
49,1200
3,1192
...,...
97956302,1
119462091,1
104240835,1


In [14]:
df.shape

(130000, 78)

In [22]:
print("Dataframe")
print(df.Label.value_counts())
# print("TEST")
# print(df_test.Label.value_counts())

Dataframe
Label
Benign     10000
LDAP       10000
MSSQL      10000
NetBIOS    10000
Portmap    10000
Syn        10000
UDP        10000
UDPLag     10000
DNS        10000
TFTP       10000
NTP        10000
SSDP       10000
SNMP       10000
Name: count, dtype: int64


In [38]:
!pip install feature-engine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.0/375.0 kB 13.1 MB/s eta 0:00:00


In [39]:
from feature_engine.discretisation import EqualWidthDiscretiser

In [40]:
num_features=df.describe().columns.tolist()

In [41]:
num_features

['Protocol',
 'FlowDuration',
 'TotalFwdPackets',
 'TotalBackwardPackets',
 'FwdPacketsLengthTotal',
 'BwdPacketsLengthTotal',
 'FwdPacketLengthMax',
 'FwdPacketLengthMin',
 'FwdPacketLengthMean',
 'FwdPacketLengthStd',
 'BwdPacketLengthMax',
 'BwdPacketLengthMin',
 'BwdPacketLengthMean',
 'BwdPacketLengthStd',
 'FlowBytes/s',
 'FlowPackets/s',
 'FlowIATMean',
 'FlowIATStd',
 'FlowIATMax',
 'FlowIATMin',
 'FwdIATTotal',
 'FwdIATMean',
 'FwdIATStd',
 'FwdIATMax',
 'FwdIATMin',
 'BwdIATTotal',
 'BwdIATMean',
 'BwdIATStd',
 'BwdIATMax',
 'BwdIATMin',
 'FwdPSHFlags',
 'BwdPSHFlags',
 'FwdURGFlags',
 'BwdURGFlags',
 'FwdHeaderLength',
 'BwdHeaderLength',
 'FwdPackets/s',
 'BwdPackets/s',
 'PacketLengthMin',
 'PacketLengthMax',
 'PacketLengthMean',
 'PacketLengthStd',
 'PacketLengthVariance',
 'FINFlagCount',
 'SYNFlagCount',
 'RSTFlagCount',
 'PSHFlagCount',
 'ACKFlagCount',
 'URGFlagCount',
 'CWEFlagCount',
 'ECEFlagCount',
 'Down/UpRatio',
 'AvgPacketSize',
 'AvgFwdSegmentSize',
 'AvgBwdS

In [23]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['Label']= le.fit_transform(df['Label'])
df['Label'].unique()

array([ 0,  2,  3,  5,  6,  9, 11, 12,  1, 10,  4,  8,  7])

In [15]:
len(df[(df['FlowDuration']>1000)])

41816

In [16]:
len(df[(df['FlowDuration']<=1000)])

88184

In [17]:
41816+88184

130000

In [27]:
len(df[(df['FlowDuration']>10000)& (df['Label'] == 0) ])

7844

### In this notebook the problem is treated as binary
The Label column typically offers the class and for some classes with multiple attacks it labels them individually.
In this notebook the first focus is the binary problem so we collapse the class label.

In [28]:
target = 'Label'

In [29]:
#Input, Target split
def xs_y(df_, targ):
    if not isinstance(targ, list):
        xs = df_[df_.columns.difference([targ])].copy()
    else:
        xs = df_[df_.columns.difference(targ)].copy()
    y = df_[targ].copy()
    return xs, y

In [30]:
conts = list(df.columns.difference([target]).values)
len(conts)

77

In [31]:
conts

['ACKFlagCount',
 'ActiveMax',
 'ActiveMean',
 'ActiveMin',
 'ActiveStd',
 'AvgBwdSegmentSize',
 'AvgFwdSegmentSize',
 'AvgPacketSize',
 'BwdAvgBulkRate',
 'BwdAvgBytes/Bulk',
 'BwdAvgPackets/Bulk',
 'BwdHeaderLength',
 'BwdIATMax',
 'BwdIATMean',
 'BwdIATMin',
 'BwdIATStd',
 'BwdIATTotal',
 'BwdPSHFlags',
 'BwdPacketLengthMax',
 'BwdPacketLengthMean',
 'BwdPacketLengthMin',
 'BwdPacketLengthStd',
 'BwdPackets/s',
 'BwdPacketsLengthTotal',
 'BwdURGFlags',
 'CWEFlagCount',
 'Down/UpRatio',
 'ECEFlagCount',
 'FINFlagCount',
 'FlowBytes/s',
 'FlowDuration',
 'FlowIATMax',
 'FlowIATMean',
 'FlowIATMin',
 'FlowIATStd',
 'FlowPackets/s',
 'FwdActDataPackets',
 'FwdAvgBulkRate',
 'FwdAvgBytes/Bulk',
 'FwdAvgPackets/Bulk',
 'FwdHeaderLength',
 'FwdIATMax',
 'FwdIATMean',
 'FwdIATMin',
 'FwdIATStd',
 'FwdIATTotal',
 'FwdPSHFlags',
 'FwdPacketLengthMax',
 'FwdPacketLengthMean',
 'FwdPacketLengthMin',
 'FwdPacketLengthStd',
 'FwdPackets/s',
 'FwdPacketsLengthTotal',
 'FwdSegSizeMin',
 'FwdURGFlag

In [32]:
X, y = xs_y(df, targ=target)

In [33]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [34]:
#x_train,y_train, x_test,y_test
len(x_train), len(y_train), len(x_test), len(y_test)

(97500, 97500, 32500, 32500)

In [36]:
print(y_train.value_counts())

Label
3     7615
9     7566
12    7530
2     7518
5     7498
0     7491
6     7481
10    7480
8     7480
7     7480
4     7477
1     7469
11    7415
Name: count, dtype: int64


In [35]:
print(y_test.value_counts())

Label
11    2585
1     2531
4     2523
8     2520
7     2520
10    2520
6     2519
0     2509
5     2502
2     2482
12    2470
9     2434
3     2385
Name: count, dtype: int64


## Ensemble OneR: 0.991 ROC_AUC score, almost perfect
The simplest extension is the average OneR model.

1. It takes all the features which actually have predictive power
2. It uses those single-feature models to predict all samples
3. The new predicted output class for a sample is the average (here unweighted) of the predicted output classes from each of the OneR models.

If the metric accepts scores or probabilities then you can pass them directly (which is the case for roc_auc).

In [ ]:
def roc_auc_score_multiclass(feature, metric=roc_auc_score, average=None, multi_class="ovr"):
    rootnode = DecisionTreeClassifier(max_depth=1, criterion='gini')
    rootnode.fit(x_train[feature].array.reshape(-1,1), y_train)
    pred_tr = rootnode.predict(x_train[feature].array.reshape(-1,1))
    pred_ts = rootnode.predict(x_test[feature].array.reshape(-1,1))
    # met = round(metric(y_train, pred_tr), 4)
    # if met > 0.5:
    #     return [feature, met, rootnode, pred_tr, pred_ts]
    # else:
    #     return [feature, met, None, [], []]

    #creating a set of all the unique classes using the actual class list
    unique_class = set(y_train)
    roc_auc_dict = {}
    for per_class in unique_class:
      #creating a list of all the classes except the current class
      other_class = [x for x in unique_class if x != per_class]

      #marking the current class as 1 and all other classes as 0
      new_actual_class = [0 if x in other_class else 1 for x in y_train]
      new_pred_class = [0 if x in other_class else 1 for x in pred_tr]

      #using the sklearn metrics method to calculate the roc_auc_score
      roc_auc =  round(metric(new_actual_class, new_pred_class, average=None, multi_class="ovr"), 7)
    return [feature, roc_auc, rootnode, pred_tr, pred_ts]

In [ ]:
import os
results = parallel(f=roc_auc_score_multiclass,items=conts, n_workers=os.cpu_count(), threadpool=False, progress=True)

In [ ]:
result_df = pd.DataFrame(data=results, columns=['feature', 'roc_auc_score', 'fitted_models', 'pred_tr', 'pred_ts']).sort_values(by='roc_auc_score', ascending=False)

In [ ]:
result_df

In [ ]:
result_df[['feature', 'roc_auc_score']].head(48)

In [ ]:
useful_features = result_df.loc[result_df['roc_auc_score'] > 0.49]
print(f"{len(useful_features)} / {len(conts)} features have separating power")

In [ ]:
useful_features

In [ ]:
x=np.vstack(useful_features['pred_tr'].to_numpy())
x,x.shape

In [ ]:
y=np.vstack(result_df['pred_ts'].to_numpy())
y,y.shape

In [ ]:
ensemble_pred_tr = np.mean(np.vstack(useful_features['pred_tr'].to_numpy()), axis=0)
ensemble_pred_tr.shape

In [ ]:
np.vstack(useful_features['pred_tr'][0]), np.vstack(useful_features['pred_tr'][0]).shape

In [ ]:
np.vstack(useful_features['pred_tr']), np.vstack(useful_features['pred_tr']).shape

In [ ]:
matrix=[np.vstack(useful_features['pred_tr'])]
matrix

In [ ]:
mt = [list(row) for row in zip(*matrix)]
mt

In [ ]:
np.bincount(np.vstack(useful_features['pred_tr'].to_numpy()))

In [ ]:
matrix = [[1, 1, 1, 1],
		[2, 2, 2, 2],
		[3, 3, 3, 3],
		[4, 4, 4, 4]]
type(matrix)
# matrix[1]

In [ ]:
transpose = [list(row) for row in zip(*matrix)]

print("Original Matrix:")
for row in matrix:
	print(row)

print("Transposed Matrix:")
for row in transpose:
	print(row)

In [ ]:
ensemble_pred_tr = np.bincount(mt).argmax()
ensemble_pred_tr, ensemble_pred_tr.shape

In [ ]:
uftr=np.vstack(useful_features['pred_tr'])

In [ ]:
uftr, uftr.shape

In [ ]:
uftr.shape

In [ ]:
uftr[0].shape

In [ ]:
ensemble_pred_tr = np.bincount(uftr).argmax()
ensemble_pred_tr, ensemble_pred_tr.shape

In [ ]:
x=np.vstack(result_df['pred_ts'].to_numpy())
x,x.shape

In [ ]:
useful_features['pred_tr'],useful_features['pred_tr'].shape

In [ ]:
useful_features['pred_tr'][0],useful_features['pred_tr'][0].shape

In [ ]:
 np.bincount(np.vstack(useful_features['pred_tr'][0].to_numpy()))

In [ ]:
ar

In [ ]:
ar[0]
ar.shape

In [ ]:
ar[0],ar[0].shape

In [ ]:
np.bincount(ar[0]).argmax()

In [ ]:
ensemble_pred_tr = np.bincount(ar).argmax()
ensemble_pred_tr.shape

In [ ]:
result_df['pred_ts'].to_numpy()

In [ ]:
l=len(result_df['pred_ts'])
l

In [ ]:
result_df['pred_ts']

In [ ]:
ar=np.vstack(result_df['pred_tr'])
l=len(ar)
l

In [ ]:
ensemble_pred_tr=list(np.bincount(ar).argmax())
ensemble_pred_tr

In [ ]:
ar=np.vstack(useful_features['pred_tr'])
ar[0].shape,ar[

In [ ]:
ept=list()
ept=np.bincount(ar).argmax()
ept

In [ ]:
ept=list()
ept=np.bincount(mt).argmax()
ept

In [ ]:
for row in mt:
    ept[i] = np.bincount(mt[row]).argmax()
    # ensemble_pred_tr[i]=np.bincount(result_df['pred_tr'][i]).argmax()
    print(ept[i])

In [ ]:
ensemble_pred_tr=list(l)
def cal_ensemble_pred_tr(ar):
    for i in range(l):
        ensemble_pred_tr[np.bincount(ar[i]).argmax()]
        print(ensemble_pred_tr[i])
cal_ensemble_pred_tr(ar)

In [ ]:
df.head(10)

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,17,40694755,5,0,1500,0,300,300,300.00000,0.000000,...,-1062718975,0.000000,0.000000,0,0,10173689.0,3.593932e+06,14671724,6369775,Benign
1,0,113871580,52,0,0,0,0,0,0.00000,0.000000,...,0,6.083334,0.668558,7,5,9489292.0,2.133550e+05,9806952,9169328,Benign
2,0,112936309,39,0,0,0,0,0,0.00000,0.000000,...,0,4.916666,0.514929,6,4,9411354.0,2.927876e+05,9869393,9048974,Benign
3,6,24944828,8,6,37,0,31,0,4.62500,10.662852,...,20,20876.000000,313.955400,21098,20654,10009953.0,3.139554e+02,10010175,10009731,Benign
4,17,20779,2,2,86,366,43,43,43.00000,0.000000,...,0,0.000000,0.000000,0,0,0.0,0.000000e+00,0,0,Benign
5,6,118074966,36,48,2474,67854,517,0,68.72222,134.701700,...,20,88838.000000,15440.384000,99756,77920,58909704.0,2.302835e+04,58925988,58893420,Benign
6,17,20568,2,2,86,118,43,43,43.00000,0.000000,...,32,0.000000,0.000000,0,0,0.0,0.000000e+00,0,0,Benign
7,17,65675,2,2,86,118,43,43,43.00000,0.000000,...,32,0.000000,0.000000,0,0,0.0,0.000000e+00,0,0,Benign
8,17,20684,2,2,86,198,43,43,43.00000,0.000000,...,20,0.000000,0.000000,0,0,0.0,0.000000e+00,0,0,Benign
9,6,90699,4,2,62,70,31,0,15.50000,17.897858,...,20,0.000000,0.000000,0,0,0.0,0.000000e+00,0,0,Benign


In [ ]:
result_df['pred_ts'][0].shape

In [ ]:
ensemble_pred_ts = np.mean(np.vstack(result_df['pred_ts'].to_numpy()), axis=0)
ensemble_pred_ts.shape

In [ ]:
ensemble_pred_tr = np.mean(np.vstack(useful_features['pred_tr'].to_numpy()), axis=0)
ensemble_pred_tr.shape

In [ ]:
ensemble_pred_ts = np.mean(np.vstack(useful_features['pred_ts'].to_numpy()), axis=0)
ensemble_pred_ts.shape

In [ ]:
ensemble_pred_ts

In [ ]:
fpr, tpr, thresholds = roc_curve(y_train, ensemble_pred_tr)
# get the best threshold
J = tpr - fpr
ix = np.argmax(J)
best_thresh = thresholds[ix]
print("Best threshold", best_thresh)

In [ ]:
print("The Ensemble OneR model (simple average)")
print("ROC-AUC", round(roc_auc_score(y_true=y_train, y_score=ensemble_pred_tr),4))
print("Precision", round(precision_score(y_true=y_train, y_pred=np.where(ensemble_pred_tr >= best_thresh, 1, 0)), 4))
print("Recall", round(recall_score(y_true=y_train, y_pred=np.where(ensemble_pred_tr >= best_thresh, 1, 0)), 4))
print("F1", round(f1_score(y_true=y_train, y_pred=np.where(ensemble_pred_tr >= best_thresh, 1, 0)), 4))

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, ensemble_pred_ts)
# get the best threshold
J = tpr - fpr
ix = np.argmax(J)
ts_best_thresh = thresholds[ix]
print("Best threshold", ts_best_thresh)

In [ ]:
print("The Ensemble OneR model (simple average)")
print("ROC-AUC", round(roc_auc_score(y_true=y_test, y_score=ensemble_pred_ts),4))
print("Precision", round(precision_score(y_true=y_test, y_pred=np.where(ensemble_pred_ts >= ts_best_thresh, 1, 0)), 4))
print("Recall", round(recall_score(y_true=y_test, y_pred=np.where(ensemble_pred_ts >= ts_best_thresh, 1, 0)), 4))
print("F1", round(f1_score(y_true=y_test, y_pred=np.where(ensemble_pred_ts >= ts_best_thresh, 1, 0)), 4))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
fig, axes = plt.subplots(5,3, figsize=(15,15))
axes = axes.flatten()
for i, tf in enumerate(result_df['feature'].head(48)):
    sns.histplot(data=df, x=tf, stat='percent', hue='Label', bins=100, log_scale=(False, True), ax=axes[i])

In [ ]:
bin_pred_tr=np.where(ensemble_pred_tr >= best_thresh, 1, 0)

In [ ]:
bin_pred_ts=np.where(ensemble_pred_ts >= ts_best_thresh, 1, 0)

In [ ]:
Final_Result_tr=df.assign(bin_pred_tr=bin_pred_tr)

In [ ]:
Final_Result_ts=df_test.assign(bin_pred_ts=bin_pred_ts)

In [ ]:
Final_Result_ts

In [ ]:
Final_Result_tr.to_csv('CIC-DDoS2019trBin.csv')

In [ ]:
Final_Result_ts.to_csv('CIC-DDoS2019tsBin.csv')

In [ ]:
def evaluate_one_feature(feature, index='', metric=roc_auc_score,average=None, multi_class="ovr"):
    rootnode = DecisionTreeClassifier(max_depth=1, criterion='gini')
    rootnode.fit(x_train[feature].array.reshape(-1,1), y_train)
    pred_tr = rootnode.predict(x_train[feature].array.reshape(-1,1))
    pred_ts = rootnode.predict(x_test[feature].array.reshape(-1,1))
    met = round(metric(y_train, pred_tr), 4)
    if met > 0.5:
        return [feature, met, rootnode, pred_tr, pred_ts]
    else:
        return [feature, met, None, [], []]

In [ ]:
evaluate_one_feature('Flow Bytes/s')

In [ ]:
import os
results = parallel(f=evaluate_one_feature,
                  items=conts, n_workers=os.cpu_count(), threadpool=False, progress=True)

In [ ]:
#result_df = pd.DataFrame(data=results, columns=['feature', 'roc_auc_score', 'fitted_models', 'pred_tr']).sort_values(by='roc_auc_score', ascending=False)
result_df = pd.DataFrame(data=results, columns=['feature', 'roc_auc_score', 'fitted_models', 'pred_tr', 'pred_ts']).sort_values(by='roc_auc_score', ascending=False)

In [ ]:
result_df

In [ ]:
result_df[['feature', 'roc_auc_score']].head(48)

In [ ]:
ensemble_pred_tr = np.mean(np.vstack(useful_features['pred_tr'].to_numpy()), axis=0)

In [ ]:
import numpy as np


# create array
x = np.array([1,2,3,4,5,1,2,1,1,1])
print("Original array:")
print(x)

print("Most frequent value in the above array:")
print(np.bincount(x).argmax())


In [ ]:
# create array
mtr = np.array([[1,2,3,4,5,1,2,1,1,1],
       [3, 4, 4, 2,3,4,5, 4, 4, 3],
       [2, 5, 3, 2,3,4,5, 5, 5, 3],
       [0, 0, 1,2,3,4,5, 2, 2, 4]])
print("Original array:")
print(mtr)
l=len(mtr)
print(l)
np.bincount(x[0]).argmax()

In [ ]:
np.bincount(mtr[0])

In [ ]:
mn = np.bincount((mtr[0]),axis=0)
mn

In [ ]:
print("Most frequent value in the above array:")
for i in range(l):
    print(np.bincount(x[i]).argmax())

In [ ]:
print("Most frequent value in the above array:")
for row in mt:
    print(np.bincount(row.values).argmax())

from sklearn import tree
bwdpkt_tree = result_df.loc[result_df.feature == 'Bwd Packets/s']['fitted_models'].values[0]
tree.plot_tree(bwdpkt_tree)

In [ ]:
#     return ens_pre_tr
# ensemble_pred_tr=cal_ensemble_pred_tr(ar)
# ensemble_pred_tr